In [1]:

import pandas as pd
import requests
import sys
def pull_data(url):


    headers = {
                                    "Host": "stats.nba.com",
                                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
                                    "Accept": "application/json, text/plain, */*",
                                    "Accept-Language": "en-US,en;q=0.5",
                                    "Accept-Encoding": "gzip, deflate, br",

                                    "Connection": "keep-alive",
                                    "Referer": "https://stats.nba.com/"
                                }
    json = requests.get(url,headers = headers).json()
    if len(json["resultSets"])== 1:

        
        data = json["resultSets"][0]["rowSet"]
        #print(data)
        columns = json["resultSets"][0]["headers"]
        #print(columns)
        
        df = pd.DataFrame.from_records(data, columns=columns)
    else:

        data = json["resultSets"]["rowSet"]
        print(data)
        #print(json)
        columns = json["resultSets"]["headers"][1]['columnNames']
        #print(columns)
        df = pd.DataFrame.from_records(data, columns=columns)
    return df

def pull_series():
    frames=[]
    for year in range(1997,2025):

        season = str(year-1)+'-'+str(year)[-2:]
        for round in range(1,5):
        
        #part3 = "&SeasonSegment=&SeasonType=Regular%20Season&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight="
            url ='https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound='+str(round)+'&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
        
            df = pull_data(url)
            url2='https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType=Advanced&Month=0&OpponentTeamID=0&Outcome=&PORound='+str(round)+'&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season='+season+'&SeasonSegment=&SeasonType=Playoffs&ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='
            df2 = pull_data(url2)
            joined_columns = set(df2.columns) - set(df.columns)

            joined_columns=list(joined_columns)
            joined_columns.append('PLAYER_ID')
            df2=df2[joined_columns]
            print(len(df))
            df=df.merge(df2,on='PLAYER_ID')
            df['year']=year
            df['round']=round
            print(len(df2))
            print(len(df))
            #print(df)
            frames.append(df)
        print(year)
    total=pd.concat(frames)
    return total
#all_series=pull_series()
#all_series.to_csv('player_series.csv',index=False)

In [2]:
all_series= pd.read_csv('player_series.csv')
all_series['TEAM']=all_series['TEAM_ABBREVIATION']
all_series['TEAM_ID']=all_series['TEAM_ID'].astype(str)

In [3]:
all_series

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,OFF_RATING_RANK,DREB_PCT_RANK,E_NET_RATING_RANK,E_OFF_RATING,OREB_PCT_RANK,FGA_PG_RANK,REB_PCT_RANK,year,round,TEAM
0,243,Aaron McKie,Aaron,1610612765,DET,24.0,5,2,3,0.400,...,150,102,62,101.3,130,103,133,1997,1,DET
1,228,Adam Keefe,Adam,1610612762,UTA,27.0,2,2,0,1.000,...,123,12,141,101.1,136,172,7,1997,1,UTA
2,673,Alan Henderson,Alan,1610612737,ATL,24.0,5,3,2,0.600,...,143,45,120,99.7,34,132,38,1997,1,ATL
3,275,Allan Houston,Allan,1610612752,NYK,26.0,3,3,0,1.000,...,29,149,63,118.6,128,26,154,1997,1,NYK
4,297,Alonzo Mourning,Alonzo,1610612748,MIA,27.0,5,3,2,0.600,...,75,21,35,107.0,32,24,16,1997,1,MIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10285,1630202,Payton Pritchard,Payton,1610612738,BOS,26.0,5,4,1,0.800,...,20,25,12,103.0,14,10,26,2024,4,BOS
10286,1630573,Sam Hauser,Sam,1610612738,BOS,26.0,5,4,1,0.800,...,10,18,8,114.1,7,11,14,2024,4,BOS
10287,1629004,Svi Mykhailiuk,Svi,1610612738,BOS,27.0,3,2,1,0.667,...,28,6,29,92.3,20,19,16,2024,4,BOS
10288,203501,Tim Hardaway Jr.,Tim,1610612742,DAL,32.0,4,1,3,0.250,...,25,19,24,97.7,20,16,25,2024,4,DAL


In [4]:
opp = pd.read_csv('opponentmatch.csv')
opp.rename(columns={'Round':'round'},inplace=True)
opp['TEAM_ID']=opp['TEAM_ID'].astype(str)
opp['OPP_ID']=opp['OPP_ID'].astype(str)

opp['round']=opp['round'].astype(int)
opp.drop(columns='TEAM',inplace=True)

In [5]:
all_series=all_series.merge(opp,on=['TEAM_ID','round','year'])



all_series.drop_duplicates(subset=['PLAYER_ID','year','round'],inplace=True)

In [6]:
ff = pd.read_csv('fourfactors.csv')
ff=ff[['TEAM_ID','OPP_TS_PCT','year']]

avg= pd.read_csv('avg_shooting.csv')

avg['year'] = avg['Season'].str.split('-',expand=True)[0].astype(int)+1

avg=avg[['year','TS%']]
avg.columns=['year','year_avg_ts']

ff=ff.merge(avg,on='year',how='left')
ff.rename(columns={'TEAM_ID':'OPP_ID'},inplace=True)
ff['OPP_ID']=ff['OPP_ID'].astype(str)
ff.drop_duplicates(subset=['year','OPP_ID'],inplace=True)

In [7]:
ff

,OPP_ID,OPP_TS_PCT,year,year_avg_ts
0,1610612737,0.509832,1997,0.535772
1,1610612738,0.575160,1997,0.535772
2,1610612766,0.532709,1997,0.535772
3,1610612741,0.509359,1997,0.535772
4,1610612739,0.533107,1997,0.535772
...,...,...,...,...
827,1610612758,0.591972,2024,0.580354
828,1610612759,0.585274,2024,0.580354
829,1610612761,0.594523,2024,0.580354
830,1610612762,0.595921,2024,0.580354


In [8]:
final_frame= all_series.merge(ff,how='left')
#temp.drop_duplicates(subset=['PLAYER_ID','year','round','OPP_ID'])
ts_col = [col for col in final_frame.columns if 'TS' in col]
ts_col = ['TS_PCT_RANK',
 'TS_PCT',
 'OPP_TS_PCT',
 'year_avg_ts']
for col in ts_col:
    final_frame[col]*=100

final_frame['rOPP_TS_PCT'] = final_frame['TS_PCT']-final_frame['OPP_TS_PCT']
final_frame['rTS_PCT']=final_frame['TS_PCT']-final_frame['year_avg_ts']
final_frame

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,REB_PCT_RANK,year,round,TEAM,OPP,OPP_ID,OPP_TS_PCT,year_avg_ts,rOPP_TS_PCT,rTS_PCT
0,243,Aaron McKie,Aaron,1610612765,DET,24.0,5,2,3,0.400,...,133,1997,1,DET,ATL,1610612737,50.983165,53.577167,-13.483165,-16.077167
1,690,Don Reid,Don,1610612765,DET,23.0,1,0,1,0.000,...,40,1997,1,DET,ATL,1610612737,50.983165,53.577167,62.616835,60.022833
2,255,Grant Hill,Grant,1610612765,DET,24.0,5,2,3,0.400,...,66,1997,1,DET,ATL,1610612737,50.983165,53.577167,-1.883165,-4.477167
3,3,Grant Long,Grant,1610612765,DET,31.0,5,2,3,0.400,...,97,1997,1,DET,ATL,1610612737,50.983165,53.577167,3.716835,1.122833
4,966,Jerome Williams,Jerome,1610612765,DET,24.0,1,0,1,0.000,...,2,1997,1,DET,ATL,1610612737,50.983165,53.577167,49.016835,46.422833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10283,1629052,Oshae Brissett,Oshae,1610612738,BOS,26.0,3,2,1,0.667,...,17,2024,4,BOS,DAL,1610612742,57.855541,58.035355,13.844459,13.664645
10284,1630202,Payton Pritchard,Payton,1610612738,BOS,26.0,5,4,1,0.800,...,26,2024,4,BOS,DAL,1610612742,57.855541,58.035355,-27.455541,-27.635355
10285,1630573,Sam Hauser,Sam,1610612738,BOS,26.0,5,4,1,0.800,...,14,2024,4,BOS,DAL,1610612742,57.855541,58.035355,15.644459,15.464645
10286,1629004,Svi Mykhailiuk,Svi,1610612738,BOS,27.0,3,2,1,0.667,...,16,2024,4,BOS,DAL,1610612742,57.855541,58.035355,-26.555541,-26.735355


In [9]:
final_frame.to_csv('playoff_series.csv',index=False)

In [12]:
final_frame[final_frame.PLAYER_NAME=='Stephen Curry']

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,REB_PCT_RANK,year,round,TEAM,OPP,OPP_ID,OPP_TS_PCT,year_avg_ts,rOPP_TS_PCT,rTS_PCT
5665,201939,Stephen Curry,Stephen,1610612744,GSW,25.0,6,4,2,0.667,...,138,2013,1,GSW,DEN,1610612743,52.827941,53.425709,8.872059,8.274291
5834,201939,Stephen Curry,Stephen,1610612744,GSW,25.0,6,2,4,0.333,...,86,2013,2,GSW,SAS,1610612759,51.547492,53.425709,-0.947492,-2.825709
6047,201939,Stephen Curry,Stephen,1610612744,GSW,26.0,7,3,4,0.429,...,162,2014,1,GSW,LAC,1610612746,52.659020,54.080929,7.240980,5.819071
6464,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,4,4,0,1.000,...,116,2015,1,GSW,NOP,1610612740,53.288080,53.385101,8.111920,8.014899
6604,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,6,4,2,0.667,...,62,2015,2,GSW,MEM,1610612763,52.632162,53.385101,3.267838,2.514899
6692,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,5,4,1,0.800,...,34,2015,3,GSW,HOU,1610612745,52.571560,53.385101,15.428440,14.614899
6729,201939,Stephen Curry,Stephen,1610612744,GSW,27.0,6,4,2,0.667,...,17,2015,4,GSW,CLE,1610612739,53.492456,53.385101,5.007544,5.114899
6823,201939,Stephen Curry,Stephen,1610612744,GSW,28.0,2,2,0,1.000,...,96,2016,1,GSW,HOU,1610612745,55.460573,54.148685,7.639427,8.951315
7004,201939,Stephen Curry,Stephen,1610612744,GSW,28.0,2,2,0,1.000,...,43,2016,2,GSW,POR,1610612757,54.243811,54.148685,8.356189,8.451315
7070,201939,Stephen Curry,Stephen,1610612744,GSW,28.0,7,4,3,0.571,...,27,2016,3,GSW,OKC,1610612760,52.408019,54.148685,8.791981,7.051315
